In [1]:
ls

 Volume in drive C is Windows
 Volume Serial Number is D62B-02C0

 Directory of C:\Users\yaboli2\cs440\mp1

02/07/2018  07:35 PM    <DIR>          .
02/07/2018  07:35 PM    <DIR>          ..
02/07/2018  06:45 PM    <DIR>          .ipynb_checkpoints
02/07/2018  06:45 PM             2,573 bigMaze.txt
02/07/2018  07:35 PM            40,132 Maze.ipynb
02/07/2018  06:45 PM            50,846 mazeSolve.ipynb
02/07/2018  06:45 PM             1,449 mediumMaze.txt
02/07/2018  06:45 PM               661 mediumSearch.txt
02/07/2018  06:45 PM               780 openMaze.txt
02/07/2018  06:45 PM               414 smallSearch.txt
02/07/2018  06:45 PM                80 testMaze.txt
02/07/2018  06:45 PM               106 tinySearch.txt
               9 File(s)         97,041 bytes
               3 Dir(s)  477,374,140,416 bytes free


In [2]:
import copy
import numpy as np
import numpy.linalg as la
import time
from collections import deque

In [3]:
mediumMaze = []
bigMaze = []
openMaze = []
testMaze = []
mediumSearch = []
smallSearch = []
tinySearch = []

## Check the Maze:

In [4]:
with open("openMaze.txt") as f:
    for line in f:
        openMaze.append(line.rstrip())
print(len(openMaze[0]), len(openMaze))
openMaze

37 20


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%                     %P            %',
 '%                     %             %',
 '%                     %             %',
 '%                     %             %',
 '%                     %             %',
 '%                     %%%%%%%%      %',
 '%                            %      %',
 '%                            %      %',
 '%                            %      %',
 '%                            %      %',
 '%                            %      %',
 '%      %%%%%%%%%%%%%%%%%            %',
 '%      %                            %',
 '%      %                            %',
 '%      %                            %',
 '%      %                            %',
 '%      %%%%                         %',
 '%        .%                         %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [5]:
with open("mediumMaze.txt") as f:
    for line in f:
        mediumMaze.append(line.rstrip())
mediumMaze

['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%P    %       %             %         %     %           %   %',
 '%%% %%% % %%%%% %%%%% %%%%% %%% %%%%%%% % %%% % %%%%%%% % %%%',
 '%   %         %   % %   % %   %         %   % %     % % %   %',
 '% %%% %%% %%% %%% % %%% % %%% % %%%%%% %%%% % %%%%% % % %%% %',
 '% %   % %   % %   %   % %                 % %   % % %       %',
 '%   %%% % % % % % % % % %%% % %%%%%%% %%% % %%% % % %%%%%%% %',
 '% %   %   % %   %   % %   % %     %   %   %     % %   %   % %',
 '% %%%%%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %',
 '%   %   % %       % % %   %   % %   % % %     %     %   % % %',
 '% % %%% % %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%',
 '% %   % %   %         % %   % %     %   % % %           %   %',
 '% %%%%% %%%%% %%%%% %%% % % % %%%%%%%%% % % %%% %%% %%% %%% %',
 '%     %       %         % % %         %   %     %     %   % %',
 '% %%% %%% %%%%% %%%%% % %%% %%% %%%%% %%%%%% %%%% %%% %%% % %',
 '%   %   

In [6]:
with open("bigMaze.txt") as f:
    for line in f:
        bigMaze.append(line.rstrip())
bigMaze

['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% %           %                       %   %     %       %   %               %  .%',
 '% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%% % %',
 '%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %     % %',
 '% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % % %%%%% %',
 '%         %   %   %   % % % %   % %       %     %       % % % %   % %   % %     %',
 '%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%% % %%% %',
 '%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %   % % % %',
 '% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% % %%% % % %',
 '%     %   %   %   % % %   %   %           %     %   %   %         %   %   % % % %',
 '%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%% % % % %',
 '%         %     %   % %   % %   %   %   %   %     % 

In [7]:
with open("testMaze.txt") as f:
    for line in f:
        testMaze.append(line.rstrip())
testMaze

['%%%%%%%%%%%',
 '%    %.   %',
 '%    %%%% %',
 '% P  %    %',
 '%     .   %',
 '%%%%%%%%%%%']

In [8]:
with open("mediumSearch.txt") as f:
    for line in f:
        mediumSearch.append(line.rstrip())
mediumSearch

['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% .  %.        %             %  .%     % %   %  %',
 '%    %%%% %%%%%% %  % % %%         %%%%  %. %%% %',
 '%    %.%    % .% %  % %  %   %   %          %   %',
 '%   .            %.     %%%% %   %% .%%% %      %',
 '% %%%%% %%%%%%  %%%% %           .%    %. %%%%%%%',
 '%.      %   .%       .% %%%  %%%%%%  % %  %   % %',
 '%%% %%  % %%%%%%% %%%%% %    %  . %% % %    %.% %',
 '%    %  %      %  %      P% %%       %   %%%% % %',
 '%.      %   %  %  %     %          %       .%   %',
 '% % %%%   .% %    %% %% %%% %% %  %.%%%%%%%%%%% %',
 '%   %    %    %              %    %             %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [9]:
with open("smallSearch.txt") as f:
    for line in f:
        smallSearch.append(line.rstrip())
smallSearch

['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%.     P .     %      .  .   %',
 '%   %%%%% %%%%%% %  % % %%   %',
 '%   %  %     %   %  % %  %  .%',
 '%     .%     .   %.   %  %%%%%',
 '%%%%% %%%% %%%  %%%%%%%      %',
 '%.               .    % %%%  %',
 '%% %%%%%%%% %%%%%%%%%%% %.   %',
 '%       %               % %%%%',
 '%       %%%%%     %.         %',
 '%.% %%%    %  %   %% %% %%%%%%',
 '%          % .%        .     %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [10]:
with open("tinySearch.txt") as f:
    for line in f:
        tinySearch.append(line.rstrip())
tinySearch

['%%%%%%%%%%',
 '%.  %   .%',
 '% %.% %% %',
 '% %   .%.%',
 '% .%P%   %',
 '%.  .  . %',
 '% %%%% %.%',
 '%.    .% %',
 '%%%%%%%%%%']

## Class & Function defined: 

In [38]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.width = None
        self.height = None
        self.end = [] # ready for multiple dots
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        self.expandedNumber = 0
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char == "%":
                    self.maze_list.append(0)
                elif char == ".":
                    self.maze_list.append("G")
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char == "P":
                    self.maze_list.append("P")
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)

    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]                
    
#   Transform the index in the raw_list to the index in the maze_list
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
    
    
#   Return the adjacant node who is unvisited and valid
    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent
    
    def dfs(self, start=(1,1), end=(59,21)):
        self.visited = []
        self.solution = []
        self.expandedNumber = 0
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append(start)
        #print("start")
        self.dfs_helper(start)
        
        
    def dfs_helper(self, current):
        self.expandedNumber += 1
        
        adjacent_nodes = self.adjacentList(current)
        
        
        if current in self.end:
            self.solution.append(current)
            #print("I get the destination!")
            return True
        
        if not adjacent_nodes:
            # Remove it from the visited list
            self.solution = self.solution[:-1]
            return
        
        for node in adjacent_nodes:
            nextStep= node
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            self.dfs_helper(nextStep)

        if self.solution[-1] not in self.end:
            self.solution = self.solution[:-1]       

        return
            
        
    def bfs(self, start):
        self.expandedNumber = 0
        self.bfsHelper(start)
       
    def bfsHelper(self,start):
        self.start = start
        self.visited = []
        
        queue = []
        parent = {}
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
        self.expandedNumber += 1
        while queue and len(self.end) != 0:
            current = queue[0]
            queue = queue[1:]
            
            if current in self.end:
                #print("I get to the destination!")
                self.end.remove(current)
                # The last step to the solution, track back
                self.solution.append(current)

                while current != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution = self.solution[::-1]
                
                #new search:              
                return self.bfsHelper(current)
                
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                self.expandedNumber += 1
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))     
        
        return True
    
      
    def countCost(self):
        self.cost = {}
        for g in self.end:
            for index, cell in enumerate(self.maze_list):
                x = index%self.width
                y = index//self.width
                if not cell:
                    self.cost[(x,y)] = np.inf
                else:
                    self.cost[(x,y)] = abs(x - g[0]) + abs(y - g[1])
    
    # Heuristics function for single dot maze
#     def heurisFunc(self, current):
#         delta_x = current[0] - self.end[0][0]
#         delta_y = current[1] - self.end[0][1]
#         return delta_x + delta_y
               
    def best_first_search(self, start=(1,1), end=(59,21)):
        if not self.cost:
            self.countCost()
        self.visited = []
        self.solution = []
        self.expandedNumber = 0
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append((start[0], start[1]))
        self.greedyHelper(start, False)
        
    # Sort the adjaceny nodes list based on Manhattan Distance
    # ** Single dots implementation **
    def greedySort(self, adjacentList):
        distance = []
        for node in adjacentList:
            distance.append(self.cost[node])
          
        # get the index in term of sorting
        sorted_idx = np.argsort(distance) 
        
        # sort the original adjacent node list, ****return a numpy array!****
        return np.array(adjacentList)[sorted_idx]               
            
        
    def greedyHelper(self, current, found=False): # Almost the same as dfs
        self.expandedNumber += 1
        print(current)
        if current in self.end:
            print("I get the destination!")
            self.solution.append(current)
            return True
        
        adjacent_nodes = self.adjacentList(current)
        if not adjacent_nodes:
            self.solution = self.solution[:-1]
            return
        
        # Sort the adjacent node list
        sorted_adj_nodes = self.greedySort(adjacent_nodes)
            
        for node in sorted_adj_nodes:
            nextStep = tuple(node)
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            if self.greedyHelper(nextStep):
                return True
            
        if self.solution[-1] not in self.end:
            self.solution = self.solution[:-1]       
        
        return
        
        
#     # Return a sorted queue, act like a priority queue
#     # Use Manhattan Distance + path cost as f
#     def ASort(self, queue):

#     # A* search for single dots
#     def AStar(self, start, end):
#         if not self.cost:
#             self.countCost()
    def manhattanDistance(self, cur, des): return abs(cur[0] - des[0]) + abs(cur[1] - des[1])


                
    
    def A_star_search(self, start):
        self.countCost()
        self.visited = []
        self.solution = []
        queue = []
        parent = {}
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append(start)
        
        while queue:
            current = queue[0]
            queue = queue[1:]
            
            if current in self.end:
                #print("I get to the destination!")
                # The last step to the solution, track back
                self.solution.append(current)

                while current != self.start:
                    self.solution.append(current)
                    current = parent[current]
                    
                self.solution.append(current)
                self.solution = self.solution[::-1]
                return True
            
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))
                
            queue = self.AStarSort(queue, current, parent)
            
        print("I can't find the destination!")
        return False
        
    def AStarSort(self, queue, current, parent):
        distance = []
        sorted_queue = []
        
        for node in queue:
            path = []
            while node != self.start:
                path.append(node)
                node = parent[node]
            distance.append(self.cost[node] + len(path))
          
        # get the index in term of sorting
        sorted_idx = np.argsort(distance) 
        sorted_arr = (np.array(queue)[sorted_idx]) 
        for node in sorted_arr:
            sorted_queue.append(tuple(node))
            
        return sorted_queue
     
    

    def drawSolution(self):
        solution = copy.deepcopy(self.raw_list)  
        for pos in self.solution:
            x, y = pos
            if self.raw_list[y][x] == ".":
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif self.raw_list[y][x] == "P":
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
        return solution

    

 ## Single-dot MazeRunner: 

In [39]:
m = maze("testMaze.txt")
#m = maze("tinySearch.txt")
#m = maze("smallSearch.txt")

# m.A_star_search(m.start)
m.best_first_search(m.start, m.end[0])
# m.dfs(m.start, m.end[0])
# m.bfs(m.start)
print(m.expandedNumber)
m.drawSolution()

(2, 3)
(3, 3)
(4, 3)
(4, 4)
(5, 4)
(6, 4)
I get the destination!
6


['%%%%%%%%%%%',
 '%    %.   %',
 '%    %%%% %',
 '% P..%    %',
 '%   ..G   %',
 '%%%%%%%%%%%']

In [13]:
m = maze("bigMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start)
%timeit m.A_star_search(start=m.start)
#%timeit m.best_first_search(start=m.start, end=m.end[0])

m.drawSolution()
#len(m.solution)

1 loop, best of 3: 354 ms per loop


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% %           %                       %   %     %       %   %               %..G%',
 '% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%%.% %',
 '%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %.....% %',
 '% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % %.%%%%% %',
 '%         %   %   %   % % % %   % %       %     %       % % % %   % %   %.%     %',
 '%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%%.% %%% %',
 '%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %...% % % %',
 '% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% %.%%% % % %',
 '%     %   %   %   % % %   %   %           %     %   %   %         %   %...% % % %',
 '%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%%.% % % %',
 '%         %     %   % %   % %   %   %   %   %     % 

In [92]:
m = maze("mediumMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start)
%timeit m.best_first_search(start=m.start, end=m.end[0])

#m.drawSolution()

10 loops, best of 3: 30.1 ms per loop


In [112]:
m = maze("openMaze.txt")
%timeit m.bfs(start=m.start)
#%timeit m.A_star_search(start=m.start)
#%timeit m.best_first_search(start=m.start, end=m.end[0])
m.drawSolution()

The slowest run took 13209.95 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.73 µs per loop


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%                     %P.......     %',
 '%                     %       .     %',
 '%                     %       .     %',
 '%                     %       .     %',
 '%                     %       .     %',
 '%                     %%%%%%%%.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%     ...................    %.     %',
 '%     .%%%%%%%%%%%%%%%%%.......     %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%%%%                         %',
 '%     ...G%                         %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

## Multi-dot MazeRunner:

In [94]:
#m = maze("tinySearch.txt")
m = maze("smallSearch.txt")
#m = maze("mediumSearch.txt")
%timeit m.bfs(start = m.start)
m.drawSolution()

The slowest run took 9776.26 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.48 µs per loop


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%G.....P.G     %......G..G...%',
 '%  .%%%%%.%%%%%%.%. % % %%  .%',
 '%  .%  % ....%  .%. % %  %  G%',
 '%  ...G%  . .G...%G   %  %%%%%',
 '%%%%%.%%%%.%%% .%%%%%%%      %',
 '%G....    ..   ..G    % %%%  %',
 '%%.%%%%%%%%.%%%%%%%%%%% %G   %',
 '% .     %  .............%.%%%%',
 '%..     %%%%%.    %G   ...   %',
 '%G% %%%    % .%   %% %%.%%%%%%',
 '%          % G%        G     %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [96]:
# m = maze("smallSearch.txt")
# m.bfs(start = m.start)
# m.drawSolution()

## Walking through the solution path...

In [97]:
#m = maze("tinySearch.txt")
#m = maze("smallSearch.txt")
#m = maze("mediumSearch.txt")
#m = maze("openMaze.txt")
m = maze("bigMaze.txt")

m.bfs(start = m.start)
#m.dfs(start = m.start, end = m.end[0])

In [98]:
index = 0
solution = copy.deepcopy(m.raw_list)  

def walking(maze):
    for pos in maze.solution[0:index]:
        x, y = pos
        if maze.raw_list[y][x] == ".":
            solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
        elif maze.raw_list[y][x] == "P":
            solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
        else:
            solution[y] = solution[y][:x] + "." + solution[y][x+1:]
            
    for i in range(len(solution)):
        print(solution[i])

In [24]:
walking(m)    
index = index + 1

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% %           %                       %   %     %       %   %               %  .%
% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%% % %
%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %     % %
% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % % %%%%% %
%         %   %   %   % % % %   % %       %     %       % % % %   % %   % %     %
%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%% % %%% %
%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %   % % % %
% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% % %%% % % %
%     %   %   %   % % %   %   %           %     %   %   %         %   %   % % % %
%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%% % % % %
%         %     %   % %   % %   %   %   %   %     %     %   %   % %     % % %   %
% %%%%%%% % %%%%

## Complexity and cost:

Big Maze:

| Search Method |  Path Cost  | Time Complexity (per loop) |
|---------------|-------------|-----------------|
|  DFS          |    236      |    39.9 ms      |
|  BFS          |    157      |    1.5 µs       |
| Greedy        |    256      |    82.7 ms      |
|   A\*         |  |  |

Medium Maze:

| Search Method |  Path Cost  | Time Complexity (per loop) |
|---------------|-------------|-----------------|
|  DFS          |           |          |
|  BFS          |           |            |
| Greedy        |           |          |
|   A\*         |  |  |

Open Maze:

| Search Method |  Path Cost  | Time Complexity (per loop) |
|---------------|-------------|-----------------|
|  DFS          |           |          |
|  BFS          |           |       |
| Greedy        |           |        |
|   A\*         |  |  |